In [8]:
import time

In [9]:
def encode_varint(x):
    ret = b''
    while True:
        ret += bytes([x & 0x7f])
        if x < 0x80:
            break
        x >>= 7
    return ret

def decode_varint(x):
    ret = 0
    for i in range(len(x)):
        ret += x[i] << (7 * i)
        if x[i] < 0x80:
            return ret

In [10]:
def put_varint(x):
    global buf
    buf += encode_varint(x)

In [11]:
def put_u8(x):
    global buf
    buf += x.to_bytes(1, "big")

def put_u16(x):
    global buf
    buf += x.to_bytes(2, 'big')

def put_u32(x):
    global buf
    buf += x.to_bytes(4, 'big')

def put_u64(x):
    global buf
    buf += x.to_bytes(8, 'big')

In [12]:
def put_bytes(x):
    global buf
    put_varint(len(x))
    buf += x

def put_string(x):
    global buf
    buf += x.encode("utf-8")
    buf += b'\0'

In [13]:
def put_time():
    global buf
    put_u64(int(time.time()))

In [14]:
def file(name, data):
    global buf
    put_u8(0x00)
    put_string(name)
    put_time()
    put_bytes(data)

In [15]:
class Folder:
    def __init__(self, name):
        self.name = name

    def __enter__(self):
        global buf
        put_u8(0x01)
        put_string(self.name)
        put_time()

    
    def __exit__(self, type, value, traceback):
        global buf
        put_u8(0xFF)

In [20]:
buf = b''

put_u32(0x766d6b70)
put_u16(0x0101)
put_u16(0x0000)

with Folder("/"):
    file("test.txt", b"test")
    file("meta.json", b'{"name": "test"}')
    
    with Folder("scenes"):
        file("meta.json", b'{"cnt": 200}')


In [21]:
buf

b'vmkp\x01\x01\x00\x00\x01/\x00\x00\x00\x00\x00b%\xfa}\x00test.txt\x00\x00\x00\x00\x00b%\xfa}\x04test\x00meta.json\x00\x00\x00\x00\x00b%\xfa}\x10{"name": "test"}\x01scenes\x00\x00\x00\x00\x00b%\xfa}\x00meta.json\x00\x00\x00\x00\x00b%\xfa}\x0c{"cnt": 200}\xff\xff'

In [22]:
with open("test.vmkp", "wb") as fp:
    fp.write(buf)